# Load data

In [1]:
import shap
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
# Load and split data
X_train, X_test, Y_train, Y_test = train_test_split(
    *shap.datasets.adult(), test_size=0.2, random_state=7)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=0)

In [3]:
# Neural network pre-processing
enc = OneHotEncoder(sparse=False)
Y_train_oh = enc.fit_transform(np.expand_dims(Y_train, -1))
Y_val_oh = enc.fit_transform(np.expand_dims(Y_val, -1))
Y_test_oh = enc.transform(np.expand_dims(Y_test, -1))

ss = StandardScaler()
ss.fit(X_train)
X_train_std = ss.transform(X_train)
X_val_std = ss.transform(X_val)
X_test_std = ss.transform(X_test)

# Set up imputer

In [4]:
import torch
import fastshap_torch
from fastshap_torch import Surrogate

In [5]:
device = torch.device('cuda', 2)
surrogate = torch.load('../models/census_surrogate.pt').eval().to(device)
num_features = X_train.shape[1]
surr = Surrogate(surrogate, num_features)

# FastSHAP

In [6]:
import torch
import torch.nn as nn
from fastshap_torch import FastSHAP

# Test samples number

In [8]:
for n_samples in (1, 4, 16, 32, 48, 64, 96):
    # Set up explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 2 * num_features)).to(device)

    # Set up FastSHAP wrapper
    fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=-1))

    # Train
    fastshap.train(
        X_train_std,
        X_val_std[:100],
        batch_size=32,
        num_samples=n_samples,
        max_epochs=200,
        eff_lambda=0,
        paired_sampling=False,
        validation_samples=128,
        validation_seed=0,
        verbose=False)

    # Print performance
    print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

    # Save model
    modifier = 'samples={}'.format(n_samples)
    explainer.cpu()
    torch.save(explainer, '../models/census_explainer {} nopenalty.pt'.format(modifier))

Best val loss = 0.00485420
Best val loss = 0.00066544
Best val loss = 0.00065956
Best val loss = 0.00065865
Best val loss = 0.00065339
Best val loss = 0.00065589
Best val loss = 0.00065450


In [9]:
for n_samples in (4, 16, 32, 48, 64, 96):
    # Set up explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 2 * num_features)).to(device)

    # Set up FastSHAP wrapper
    fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=-1))

    # Train
    fastshap.train(
        X_train_std,
        X_val_std[:100],
        batch_size=32,
        num_samples=n_samples,
        max_epochs=200,
        eff_lambda=0,
        paired_sampling=True,
        validation_samples=128,
        validation_seed=0,
        verbose=False)

    # Print performance
    print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

    # Save model
    modifier = 'paired_samples={}'.format(n_samples)
    explainer.cpu()
    torch.save(explainer, '../models/census_explainer {} nopenalty.pt'.format(modifier))

Best val loss = 0.00065687
Best val loss = 0.00065401
Best val loss = 0.00065606
Best val loss = 0.00065294
Best val loss = 0.00065200
Best val loss = 0.00065038


# Test other parameters

In [11]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * num_features)).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=-1))

# Train
fastshap.train(
    X_train_std,
    X_val_std[:100],
    batch_size=32,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=0,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nopenalty'
explainer.cpu()
torch.save(explainer, '../models/census_explainer {}.pt'.format(modifier))

Best val loss = 0.00065215


In [12]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * num_features)).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization=None, link=nn.Softmax(dim=-1))

# Train
fastshap.train(
    X_train_std,
    X_val_std[:100],
    batch_size=32,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0.1,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=0,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nonormalization'
explainer.cpu()
torch.save(explainer, '../models/census_explainer {}.pt'.format(modifier))

Best val loss = 0.00051005


In [13]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * num_features)).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization=None, link=nn.Softmax(dim=-1))

# Train
fastshap.train(
    X_train_std,
    X_val_std[:100],
    batch_size=32,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=0,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nopenalty nonormalization'
explainer.cpu()
torch.save(explainer, '../models/census_explainer {}.pt'.format(modifier))

Best val loss = 0.00049677
